In [1]:
import torch

# layer 비교하기 traffic 데이터를 가져오는 식으로 비교
from layers.Embed import *


In [2]:
# 관련 패키지 불러오기

import torch.nn as nn
import torch.nn.functional as F
from layers.Transformer_EncDec import Encoder, EncoderLayer
from layers.SelfAttention_Family import FullAttention, AttentionLayer
import numpy as np

In [3]:
# 변수 정리

seq_len = 96 # c_in으로 
label_len = 48
pred_len = 48
e_layers = 2
d_layers = 1
factor = 3
enc_in = 321
dec_in = 321
d_model = 512
batch_size = 32
c_out = 321
des = 'Exp'
learning_rate = 0.001
freq = 'h' # 입력빈도 - 세팅을 일일이 해야 하는 부분
dropout = 0.1
embed = 'timeF' # timeF, fixed, learned
class_strategy = 'projection'

In [4]:
enc_transformer_embedding_0 = DataEmbedding(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

In [5]:
enc_transformer_embedding_0

DataEmbedding(
  (value_embedding): TokenEmbedding(
    (tokenConv): Conv1d(96, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
  )
  (position_embedding): PositionalEmbedding()
  (temporal_embedding): TimeFeatureEmbedding(
    (embed): Linear(in_features=4, out_features=512, bias=False)
  )
  (dropout): Dropout(p=0.1, inplace=False)
)

In [6]:
enc_transformer_embedding_1 = DataEmbedding_inverted(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

In [7]:
enc_transformer_embedding_1

DataEmbedding_inverted(
  (value_embedding): Linear(in_features=96, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [8]:
enc_transformer_embedding_2 = DataEmbedding_inverted_TCN(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

conv2 shape torch.Size([236, 236, 2])
conv2 shape torch.Size([372, 372, 2])
conv2 shape torch.Size([512, 512, 2])


In [9]:
enc_transformer_embedding_2

DataEmbedding_inverted_TCN(
  (value_embedding): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(96, 236, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.05, inplace=False)
        (conv2): Conv1d(236, 236, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.05, inplace=False)
        (net): Sequential(
          (0): Conv1d(96, 236, kernel_size=(2,), stride=(1,), padding=(1,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.05, inplace=False)
          (4): Conv1d(236, 236, kernel_size=(2,), stride=(1,), padding=(1,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.05, inplace=False)
        )
        (downsample): Conv1d(96, 236, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1):

In [10]:
dummy_input = torch.randn(batch_size, enc_in, seq_len)

In [10]:
import pandas as pd

obj = 'electricity'
dataset_name = f'./dataset/{obj}/{obj}.csv'

df_base = pd.read_csv(dataset_name, encoding='utf8')


In [11]:
df_cut = df_base.drop(columns=['date'])
df_cut_list = []
for j in range(batch_size):
    df_cut_list.append(df_cut.iloc[j:seq_len+j, :])

In [12]:
df_cut_list

[       0      1      2      3      4       5     6       7      8      9  ...  \
 0   14.0   69.0  234.0  415.0  215.0  1056.0  29.0   840.0  226.0  265.0  ...   
 1   18.0   92.0  312.0  556.0  292.0  1363.0  29.0  1102.0  271.0  340.0  ...   
 2   21.0   96.0  312.0  560.0  272.0  1240.0  29.0  1025.0  270.0  300.0  ...   
 3   20.0   92.0  312.0  443.0  213.0   845.0  24.0   833.0  179.0  211.0  ...   
 4   22.0   91.0  312.0  346.0  190.0   647.0  16.0   733.0  186.0  179.0  ...   
 ..   ...    ...    ...    ...    ...     ...   ...     ...    ...    ...  ...   
 91  41.0  126.0   16.0  940.0  409.0  1547.0  28.0  1837.0  434.0  336.0  ...   
 92  27.0  136.0   20.0  906.0  349.0  1614.0  32.0  1595.0  405.0  338.0  ...   
 93  63.0  120.0  121.0  832.0  368.0  1351.0  23.0  1487.0  366.0  319.0  ...   
 94  55.0  109.0  332.0  727.0  401.0  1339.0  21.0  1403.0  322.0  311.0  ...   
 95  46.0   98.0  327.0  608.0  358.0  1189.0  18.0  1187.0  297.0  275.0  ...   
 
        311   

In [13]:
elec_input = np.array([df_cut_list[j].values for j in range(batch_size)]).astype('float')

In [14]:
elec_input = torch.tensor(elec_input, dtype=torch.float)

In [11]:
dummy_input


tensor([[[ 0.1011,  1.4886, -0.3352,  ..., -0.2612,  0.1785, -1.0775],
         [ 0.4506,  0.6834, -0.2184,  ...,  0.3832, -2.1318, -0.0323],
         [-1.1829, -0.5971, -0.9591,  ..., -1.8489,  0.6979,  1.5031],
         ...,
         [-1.7059, -0.0709,  0.4251,  ...,  1.0579,  0.5637,  0.1422],
         [ 0.6427,  0.4494,  0.4432,  ...,  0.3454, -0.2712, -0.5893],
         [ 3.2618,  0.8585,  0.2424,  ..., -0.3707, -1.5716, -0.2592]],

        [[-0.1825,  1.2955,  0.0864,  ...,  0.3704,  1.2158, -0.3402],
         [ 0.2949,  0.3406,  0.5457,  ...,  0.6796, -0.0885, -1.6989],
         [ 1.1521,  1.0768, -0.8525,  ...,  0.4093,  1.3842, -1.7188],
         ...,
         [-0.5904, -0.0937, -1.6161,  ...,  2.0701, -0.1695,  1.2379],
         [ 1.6437,  0.6973, -2.4861,  ..., -1.1352,  0.5089, -0.9791],
         [ 0.2952, -0.9201, -0.0307,  ...,  0.1292,  0.3065,  0.5407]],

        [[ 0.6436,  0.4255, -0.7122,  ...,  1.1663,  0.7979,  0.4631],
         [ 0.1742,  0.7437, -0.0831,  ...,  0

In [15]:
elec_input.size()

torch.Size([32, 96, 321])

In [16]:
output_20 = enc_transformer_embedding_2(elec_input, None)

In [17]:
output_20.size()

torch.Size([32, 321, 512])

In [18]:
output_20

tensor([[[1.4423e-01, 3.5103e-01, 0.0000e+00,  ..., 8.3481e-02,
          0.0000e+00, 0.0000e+00],
         [3.1255e-01, 1.0240e+00, 0.0000e+00,  ..., 3.3022e-01,
          0.0000e+00, 0.0000e+00],
         [4.6875e-01, 1.1158e+00, 0.0000e+00,  ..., 1.3868e-01,
          0.0000e+00, 3.3649e-02],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 6.6353e+01,
          0.0000e+00, 0.0000e+00],
         [2.7360e+02, 0.0000e+00, 0.0000e+00,  ..., 1.6045e+02,
          0.0000e+00, 5.6872e+01],
         [0.0000e+00, 4.7446e+02, 0.0000e+00,  ..., 2.6263e+01,
          0.0000e+00, 5.0159e+02]],

        [[7.8920e-02, 0.0000e+00, 0.0000e+00,  ..., 7.6965e-03,
          0.0000e+00, 0.0000e+00],
         [4.2101e-01, 7.4649e-01, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [3.1090e-02, 1.1429e+00, 0.0000e+00,  ..., 6.0041e-02,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 7.2307e+02, 0.0000e+00,  ..., 0.0000e+00,
          1.112

In [19]:
output_10 = enc_transformer_embedding_1(elec_input, None)

In [20]:
output_10.size()

torch.Size([32, 321, 512])

In [21]:
output_10

tensor([[[-6.8347e+01,  0.0000e+00, -4.3860e+01,  ...,  2.5942e+01,
           3.6815e+01,  1.8246e+01],
         [-1.1846e+02,  8.3842e+01, -1.6491e+01,  ...,  9.1207e+01,
           6.3337e+01, -2.4101e-01],
         [ 9.6166e+00,  5.9963e+01,  4.0222e+01,  ...,  3.0200e+01,
          -1.5836e+02, -3.4409e+01],
         ...,
         [-2.3433e+03,  1.2740e+03, -1.0484e+02,  ...,  1.8971e+03,
           3.3997e+02, -3.1282e+02],
         [-0.0000e+00,  2.5458e+02, -4.9168e+01,  ...,  3.1297e+02,
           3.0927e+02,  4.4188e+01],
         [-3.3761e+03,  2.1007e+03, -6.0642e+02,  ...,  2.8444e+03,
           9.4847e+02, -1.2630e+02]],

        [[-5.9484e+01,  2.1529e+01, -0.0000e+00,  ...,  3.9731e+01,
           4.1455e+01,  1.1361e+01],
         [-0.0000e+00,  8.8074e+01, -1.1516e+01,  ...,  7.5522e+01,
           6.9165e+01, -8.2727e-01],
         [ 3.5718e+01,  0.0000e+00, -3.0526e+00,  ...,  8.7796e+01,
          -0.0000e+00, -2.3956e+01],
         ...,
         [-2.2390e+03,  0

In [25]:
num_small_elements_20 = torch.sum(abs(output_20) < 0.0001).item()
num_small_elements_10 = torch.sum(abs(output_10) < 0.0001).item()

num_small_elements_20, num_small_elements_10, 32*321*512

(2090333, 523873, 5259264)

In [53]:
# temporal block size
temporal_conv_net_ex = TemporalConvNet(seq_len, [d_model]*3)

In [54]:
output_ex = temporal_conv_net_ex(elec_input)

In [56]:
output_ex.size()

torch.Size([32, 512, 321])

In [18]:
output_temporal = temporal_block1(dummy_input_0)

In [19]:
output_temporal.size()

torch.Size([32, 512, 321])

In [38]:
output_temporal = temporal_block(dummy_input)

RuntimeError: Given groups=1, weight of size [512, 96, 2], expected input[32, 321, 96] to have 96 channels, but got 321 channels instead

In [73]:
output_temporal

tensor([[[2.9887e-02, 1.2409e-01, 1.7761e-01,  ..., 6.5624e-02,
          9.7772e-03, 0.0000e+00],
         [0.0000e+00, 2.5196e-01, 1.3163e-02,  ..., 4.3740e-02,
          1.2417e-02, 2.1450e-02],
         [1.3100e-01, 1.3373e-01, 5.9980e-02,  ..., 0.0000e+00,
          7.0686e-03, 0.0000e+00],
         ...,
         [7.1469e-02, 0.0000e+00, 8.3666e-02,  ..., 3.7311e-03,
          1.3744e-02, 3.9985e-03],
         [0.0000e+00, 0.0000e+00, 1.4286e-02,  ..., 1.8574e-01,
          3.7800e-02, 0.0000e+00],
         [1.3890e-01, 8.2674e-02, 1.6861e-01,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 2.3342e-01, 1.2673e-01,  ..., 1.3788e-01,
          0.0000e+00, 7.7979e-03],
         [9.6090e-02, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          8.7233e-03, 5.6930e-02],
         [1.1677e-01, 0.0000e+00, 5.1605e-02,  ..., 3.8304e-02,
          2.1290e-02, 0.0000e+00],
         ...,
         [1.2013e-01, 2.0939e-01, 8.7630e-02,  ..., 0.0000e+00,
          9.215

In [93]:
output_temporal.size()

torch.Size([32, 512, 98])

In [20]:
temporal_conv_net = TemporalConvNet1(96, [512, 512, 512])

In [21]:
output_temporal_conv_net = temporal_conv_net(dummy_input_0)

In [77]:
output_temporal_conv_net

tensor([[[0.0249, 0.0278, 0.0253,  ..., 0.0000, 0.0185, 0.0000],
         [0.0203, 0.0000, 0.1856,  ..., 0.0029, 0.0084, 0.0009],
         [0.0361, 0.1856, 0.0498,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0266, 0.0214, 0.1779,  ..., 0.0000, 0.0102, 0.0115],
         [0.1257, 0.0316, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.1381, 0.0740, 0.1175,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.1560, 0.0338, 0.0129,  ..., 0.0168, 0.0184, 0.0190],
         [0.0000, 0.0776, 0.2084,  ..., 0.0044, 0.0059, 0.0027],
         [0.0862, 0.0733, 0.1529,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0796, 0.1283, 0.2700,  ..., 0.0071, 0.0089, 0.0077],
         [0.0000, 0.0000, 0.0619,  ..., 0.0000, 0.0000, 0.0000],
         [0.0028, 0.0601, 0.1972,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.1644, 0.0188, 0.0992,  ..., 0.0239, 0.0170, 0.0170],
         [0.1283, 0.0000, 0.0668,  ..., 0.0013, 0.0049, 0.0030],
         [0.3076, 0.1029, 0.0309,  ..., 0.0000, 0.0000, 0.

In [22]:
output_temporal_conv_net.size()

torch.Size([32, 512, 321])

In [23]:
tcn_res = TCN(96, 48, [512]*3, 2, 0.2)

In [24]:
tcn_res

TCN(
  (tcn): TemporalConvNet1(
    (network): Sequential(
      (0): TemporalBlock1(
        (conv1): Conv1d(96, 512, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (net): Sequential(
          (0): Conv1d(96, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.2, inplace=False)
          (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.2, inplace=False)
        )
        (downsample): Conv1d(96, 512, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock1(
        (conv1): Conv1d(512, 512, kernel_size=(2,), 

In [25]:
tcn_res(dummy_input_0)

tensor([[-3.8015, -3.6434, -3.8423,  ..., -3.7777, -3.9636, -3.9175],
        [-3.8019, -3.6425, -3.8388,  ..., -3.8389, -4.0670, -3.8083],
        [-3.9327, -3.5663, -3.6373,  ..., -3.6391, -4.0712, -3.9037],
        ...,
        [-3.7439, -3.7151, -3.6944,  ..., -3.8529, -3.9892, -4.0773],
        [-3.8571, -3.6266, -3.7591,  ..., -3.8624, -3.8717, -3.8816],
        [-3.6148, -3.7586, -3.8465,  ..., -3.7602, -3.9648, -4.0417]],
       grad_fn=<LogSoftmaxBackward0>)

In [26]:
tcn_res(dummy_input_0).size()

torch.Size([32, 48])

In [31]:
tmp_conv_net = TemporalConvNet(seq_len, [d_model]*3, 2, 0.005)

In [32]:
tmp_conv_net(dummy_input_0).size()

torch.Size([32, 96, 321]) 96
Input size: torch.Size([32, 96, 321])
torch.Size([32, 512, 323]) 512
Input size: torch.Size([32, 512, 323])
torch.Size([32, 512, 327]) 512
Input size: torch.Size([32, 512, 327])


torch.Size([32, 512, 335])

In [25]:
enc_transformer_embedding_2 = DataEmbedding_inverted_TCN(seq_len, d_model, embed, freq, dropout) # DataEmbedding -> d_model이 반드시 짝수

In [28]:
enc_transformer_embedding_2

DataEmbedding_inverted_TCN(
  (value_embedding): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(96, 512, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (net): Sequential(
          (0): Conv1d(96, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (1): ReLU()
          (2): Dropout(p=0.2, inplace=False)
          (3): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (4): ReLU()
          (5): Dropout(p=0.2, inplace=False)
        )
        (downsample): Conv1d(96, 512, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(512, 512, kernel_size=(2,), stride=(1,), paddi

In [30]:
enc_transformer_embedding_2(dummy_input_0, None)

torch.Size([32, 321, 96])


tensor([[[ 0.1124,  1.6540, -0.3725,  ..., -0.0000,  0.1983, -1.1973],
         [ 0.5006,  0.7594, -0.2427,  ...,  0.4258, -0.0000, -0.0359],
         [-1.3144, -0.6634, -1.0657,  ..., -2.0543,  0.7755,  1.6701],
         ...,
         [-1.8955, -0.0788,  0.4723,  ...,  1.1754,  0.0000,  0.1580],
         [ 0.7141,  0.4993,  0.4925,  ...,  0.3838, -0.3013, -0.6548],
         [ 3.6243,  0.9539,  0.2693,  ..., -0.4118, -1.7462, -0.2880]],

        [[-0.2028,  1.4394,  0.0960,  ...,  0.4116,  1.3508, -0.3780],
         [ 0.3277,  0.3785,  0.6064,  ...,  0.7551, -0.0983, -1.8877],
         [ 1.2801,  1.1964, -0.9473,  ...,  0.4548,  1.5380, -1.9097],
         ...,
         [-0.6560, -0.1041, -0.0000,  ...,  2.3001, -0.1883,  1.3754],
         [ 0.0000,  0.7748, -0.0000,  ..., -1.2613,  0.5654, -1.0879],
         [ 0.3280, -1.0223, -0.0341,  ...,  0.1436,  0.3406,  0.6007]],

        [[ 0.0000,  0.4728, -0.7913,  ...,  1.2959,  0.8865,  0.0000],
         [ 0.1936,  0.8263, -0.0924,  ...,  1

In [ ]:
TemporalConvNet(c_in, [d_model]*3) 

In [57]:
import numpy as np

In [59]:
np.linspace(4,12,4)

array([ 4.        ,  6.66666667,  9.33333333, 12.        ])